In [2]:
import numpy
from helper import l2_norm, poisson_2d_jacobi, poisson_solution

In [3]:
#parameters
ny, nx = 101, 101
xmin, xmax = 0.0, 1.0
ymin, ymax = -0.5, 0.5
Lx = xmax - xmin
Ly = ymax - ymin
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x,y)

b = (-2.0 * (numpy.pi / Lx) * (numpy.pi / Ly) *
     numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly))

p0 = numpy.zeros((ny, nx))

p_exact = poisson_solution(x, y, Lx, Ly)

In [5]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    
    def A(p):
        # apply laplacian operator to p
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)
    Ar = numpy.zeros_like(p)
    conv = []
    diff = rtol + 1
    ite = 0
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p) # residual
        Ar[1:-1, 1:-1] = A(r)   # laplacian of residual
        alpha = numpy.sum(r * r) / numpy.sum(r * Ar) # step size
        p = pk + alpha * r # update solution
        #boundary conditions!...are automatically enforced
        #compute relative L2-norm of difference
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [9]:
p, ites, conv_sd = poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-10)
print('ites =', ites, ' conv_sd =', conv_sd[-1])

ites = 2  conv_sd = 1.3307695446303778e-16


In [10]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        # used to apply laplacian to stuff!
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p) # initial residual
    Ad = numpy.zeros_like(p)
    conv = []
    diff = rtol +1
    ite = 0
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p) #compute initial residual
    d = r.copy() # initial search direction to be the residual
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk = r.copy()
        Ad[1:-1, 1:-1] = A(d) # laplacian of search direction
        alpha = numpy.sum(r * r) / numpy.sum(d * Ad) # step size
        p = pk + alpha * d # update solution!!
        r = rk - alpha * Ad # update residual
        #update search direction
        beta = numpy.sum(r * r) / numpy.sum(rk * rk)
        d = r + beta * d
        #boundary conditions automatically enforced
        # compute relative L2-norm of difference
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [11]:
p, ites, conv_cg = poisson_2d_conjugate_gradient(p0, b, dx, dy,
                                                 maxiter=20000,
                                                 rtol=1e-10)
print('ites =', ites, ' conv_sd =', conv_sd[-1])

ites = 2  conv_sd = 1.3307695446303778e-16


In [12]:
# compute relative L2-norm of the error
l2_norm(p, p_exact)

8.225076220929585e-05

In [13]:
p, ites, conv_jacobi = poisson_2d_jacobi(p0, b, dx, dy,
                                         maxiter=40000,
                                         rtol=1e-10)
print('Jacobi relaxation: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_jacobi[-1]))

Jacobi relaxation: 31227 iterations to reach a relative difference of 9.997923503623598e-11


In [14]:
#modify the source term to make some more difficult solutions!!!!!
b = (numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly) +
     numpy.sin(6.0 * numpy.pi * X / Lx) * 
     numpy.sin(6.0 * numpy.pi * Y / Ly))

In [15]:
maxiter, rtol = 40000, 1e-10
p, ites, conv = poisson_2d_jacobi(p0, b, dx, dy,
                                  maxiter=maxiter, rtol=rtol)
print(ites)
p, ites, conv = poisson_2d_steepest_descent(p0, b, dx, dy,
                                            maxiter=maxiter,
                                            rtol=rtol)
print(ites)
p, ites, conv = poisson_2d_conjugate_gradient(p0, b, dx, dy,
                                              maxiter=maxiter,
                                              rtol=rtol)
print(ites)

31226
27059
3


Clearly the CG method is the best one...thats the point I think